In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.model_selection import cross_val_predict, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV

In [56]:
data = pd.read_csv('DATA HP FIXX.csv', sep=';', encoding='latin1')

In [57]:
data

,Name,Details,Conditions,current_price,old_price,Release Date,Camera,Selfie,Screen Size,Resolution,SIM Card,Sensors,Network,Height,Width,Depth,Weight
0,Reno6,128 GB - 8 GB - Stellar Black,Condition: Good,"RpÊ1,900,000","RpÊ5,199,000",2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,6.4 inches,1080 x 2400 pixels,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,159.1 mm,73.3 mm,7.8 mm,173 g
1,Reno6,128 GB - 8 GB - Stellar Black,Condition: Good,"RpÊ1,900,000","RpÊ5,199,000",2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,6.4 inches,1080 x 2400 pixels,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,159.1 mm,73.3 mm,7.8 mm,173 g
2,Reno6,128 GB - 8 GB - Stellar Black,Condition: Good,"RpÊ1,900,000","RpÊ5,199,000",2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,6.4 inches,1080 x 2400 pixels,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,159.1 mm,73.3 mm,7.8 mm,173 g
3,Reno6,128 GB - 8 GB - Stellar Black,Condition: Good,"RpÊ1,900,000","RpÊ5,199,000",2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,6.4 inches,1080 x 2400 pixels,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,159.1 mm,73.3 mm,7.8 mm,173 g
4,Reno6,128 GB - 8 GB - Stellar Black,Condition: Good,"RpÊ1,900,000","RpÊ5,199,000",2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,6.4 inches,1080 x 2400 pixels,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,159.1 mm,73.3 mm,7.8 mm,173 g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286,iPhone 15 Pro Max,1 TB - Black Titanium,Condition: Very Good,"RpÊ28,110,000","RpÊ34,000,000","2023, September 22","48 MP dual pixel PDAF, sensor-shift OIS + 12 M...","12 MP f/1.9, 23mm (wide), 1/3.6"""", PDAF",6.7 inches,"1290 x 2796 pixels, 19.5",Nano-SIM and eSIM,"Face ID, accelerometer, gyro, proximity, compa...",GSM / CDMA / HSPA / EVDO / LTE / 5G,NaN,NaN,NaN,201 g (7.09 oz)
1287,iPhone 15 Pro,1 TB - Blue Titanium,Condition: New,"RpÊ28,399,000","RpÊ32,000,000","2023, September 22","48 MP dual pixel PDAF, sensor-shift OIS + 12 M...","12 MP f/1.9, 23mm (wide), 1/3.6"""", PDAF","6.1 inches, 91.3 cm2","1179 x 2556 pixels, 19.5",Nano-SIM and eSIM,"Face ID, accelerometer, gyro, proximity, compa...",GSM / CDMA / HSPA / EVDO / LTE / 5G,NaN,NaN,NaN,201 g (7.09 oz)
1288,iPhone 15 Pro Max,512 GB - Blue Titanium,Condition: New,"RpÊ28,430,000","RpÊ30,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1289,iPhone 15 Pro,1 TB - Blue Titanium,Condition: Mint,"RpÊ28,950,000","RpÊ32,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data = data.drop(columns=['Screen Size', 'Resolution', 'Height', 'Width', 'Depth', 'Weight'])

In [12]:
# Membagi kolom 'Details' dan mengambil bagian pertama sebagai 'Storage'
data['Storage'] = data['Details'].str.split(' - ').str[0]

# Menghapus bagian yang telah diambil dari kolom 'Details'
data['Details'] = data['Details'].str.replace(r'^\S+ \S+ - ', '', regex=True)

In [13]:
data1 = data.dropna()
data1.isnull().sum()

Name             0
Details          0
Conditions       0
current_price    0
old_price        0
Release Date     0
Camera           0
Selfie           0
SIM Card         0
Sensors          0
Network          0
Storage          0
dtype: int64

In [14]:
data1.duplicated().sum()
data1 = data1.drop_duplicates()

In [22]:
# remove RpÊ1,900,000 unnecessary character
data1['current_price'] = data1['current_price'].str.replace('Ê', ' ')

# remove RpÊ1,900,000 unnecessary character
data1['old_price'] = data1['old_price'].str.replace('Ê', ' ')

In [23]:
# Convert 'current_price' from string to numeric
data1['Selling price'] = data1['current_price'].replace({'Rp': '', ',': ''}, regex=True).astype(float)

# Convert 'old_price' from string to numeric
data1['Present price'] = data1['old_price'].replace({'Rp': '', ',': ''}, regex=True).astype(float)

# Drop the original columns
data1 = data1.drop(columns=['current_price', 'old_price'])

In [24]:
# Outlier Detection menggunakan metode IQR dan Winsorization
def winsorize(series):
    # Menghitung Q1, Q3, dan IQR
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    # Menghitung batas bawah dan atas berdasarkan IQR
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    # Transformasi outliers dengan menggantinya ke batas bawah atau atas
    return series.apply(lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x)

# Copy data original untuk transformasi outliers
data2 = data1.copy()

# Daftar kolom numerik yang ingin di-winsorize
numeric_columns = ['Selling price', 'Present price']

# Menerapkan winsorize pada kolom numerik
for col in numeric_columns:
    data2[col] = winsorize(data1[col])

data2.head()

,Name,Details,Conditions,Release Date,Camera,Selfie,SIM Card,Sensors,Network,Storage,Selling price,Present price
0,Reno6,8 GB - Stellar Black,Condition: Good,2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,128 GB,1900000.0,5199000.0
23,Reno6,8 GB - Stellar Black,Condition: Mint,2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,128 GB,1900000.0,5199000.0
36,Reno6,8 GB - Aurora,Condition: Mint,2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,128 GB,1900000.0,5199000.0
53,iPhone 11,Purple,Condition: Fair,2019,Dual 12MP camera,12MP,Nano-SIM & eSIM,"Face ID, Barometer, Three-axis gyro, Accelerom...",GSM/CDMA/LTE/HSPA/EVDO,128 GB,6500000.0,14499000.0
54,Redmi 9A,2 GB - Carbon Gray,Condition: Good,2020,13 MP (wide),5 MP,"Dual SIM (Nano-SIM, dual stand-by)","Accelerometer, proximity",GSM/HSPA/LTE,32 GB,750000.0,2999000.0


In [25]:
data3 = data2.copy()

# Ensure 'Storage' column is of string type before applying string methods
data3['Storage'] = data3['Storage'].astype(str)

# Replace 'GB' and 'TB' with empty string and convert to numeric
data3['Storage'] = data3['Storage'].str.replace('GB', '').str.replace('TB', '').astype(float)

# Handle 'TB' values (assuming 1 TB = 1024 GB)
data3.loc[data3['Storage'] <= 10, 'Storage'] = data3.loc[data3['Storage'] <= 10, 'Storage'] * 1024
data3

,Name,Details,Conditions,Release Date,Camera,Selfie,SIM Card,Sensors,Network,Storage,Selling price,Present price
0,Reno6,8 GB - Stellar Black,Condition: Good,2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,128.0,1900000.0,5199000.0
23,Reno6,8 GB - Stellar Black,Condition: Mint,2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,128.0,1900000.0,5199000.0
36,Reno6,8 GB - Aurora,Condition: Mint,2021,"64 MP (wide), 8 MP (ultrawide), 2 MP (macro), ...",44 MP,"Dual SIM (Nano-SIM, dual stand-by)","Fingerprint (under display, optical), accelero...",GSM / HSPA / LTE,128.0,1900000.0,5199000.0
53,iPhone 11,Purple,Condition: Fair,2019,Dual 12MP camera,12MP,Nano-SIM & eSIM,"Face ID, Barometer, Three-axis gyro, Accelerom...",GSM/CDMA/LTE/HSPA/EVDO,128.0,6500000.0,14499000.0
54,Redmi 9A,2 GB - Carbon Gray,Condition: Good,2020,13 MP (wide),5 MP,"Dual SIM (Nano-SIM, dual stand-by)","Accelerometer, proximity",GSM/HSPA/LTE,32.0,750000.0,2999000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1283,iPhone 15 Pro Max,White Titanium,Condition: New,"2023, September 22","48 MP dual pixel PDAF, sensor-shift OIS + 12 M...","12 MP f/1.9, 23mm (wide), 1/3.6"""", PDAF",Nano-SIM and eSIM,"Face ID, accelerometer, gyro, proximity, compa...",GSM / CDMA / HSPA / EVDO / LTE / 5G,512.0,26449000.0,30000000.0
1284,iPhone 15 Pro Max,Black Titanium,Condition: New,"2023, September 22","48 MP dual pixel PDAF, sensor-shift OIS + 12 M...","12 MP f/1.9, 23mm (wide), 1/3.6"""", PDAF",Nano-SIM and eSIM,"Face ID, accelerometer, gyro, proximity, compa...",GSM / CDMA / HSPA / EVDO / LTE / 5G,512.0,26449000.0,30000000.0
1285,iPhone 15 Pro,Black Titanium,Condition: Mint,"2023, September 22","48 MP dual pixel PDAF, sensor-shift OIS + 12 M...","12 MP f/1.9, 23mm (wide), 1/3.6"""", PDAF",Nano-SIM and eSIM,"Face ID, accelerometer, gyro, proximity, compa...",GSM / CDMA / HSPA / EVDO / LTE / 5G,1024.0,26595000.0,32000000.0
1286,iPhone 15 Pro Max,Black Titanium,Condition: Very Good,"2023, September 22","48 MP dual pixel PDAF, sensor-shift OIS + 12 M...","12 MP f/1.9, 23mm (wide), 1/3.6"""", PDAF",Nano-SIM and eSIM,"Face ID, accelerometer, gyro, proximity, compa...",GSM / CDMA / HSPA / EVDO / LTE / 5G,1024.0,26595000.0,34000000.0


In [26]:
unique_val = data3.select_dtypes(include=['object']).nunique()
unique_val = unique_val[unique_val <= 6].index #diambil nunique <= 3 agar kolom hasil encoding tidak terlalu banyak
unique_val

Index(['Conditions'], dtype='object')

In [27]:
#semua kolom objek yang bukan di unique_val
non_unique_val = [col for col in data3.select_dtypes(include=['object']).columns if col not in unique_val]
non_unique_val[0:]

['Name',
 'Details',
 'Release Date',
 'Camera',
 'Selfie',
 'SIM Card',
 'Sensors',
 'Network']

In [28]:
numerics = data3.select_dtypes(exclude=['object']).drop(columns=['Selling price'])
numerics

,Storage,Present price
0,128.0,5199000.0
23,128.0,5199000.0
36,128.0,5199000.0
53,128.0,14499000.0
54,32.0,2999000.0
...,...,...
1283,512.0,30000000.0
1284,512.0,30000000.0
1285,1024.0,32000000.0
1286,1024.0,34000000.0


In [29]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.frequency_dict = {}
        for col in X.columns:
            self.frequency_dict[col] = X[col].value_counts().to_dict()
        return self

    def transform(self, X):
        X_encoded = X.copy()
        for col in X.columns:
            X_encoded[col] = X_encoded[col].map(self.frequency_dict[col])
        return X_encoded

    # Add the set_output method
    def set_output(self, transform):
        """This method is required for compatibility with set_output(transform='pandas') in ColumnTransformer."""
        return self

# Inisialisasi transformers
frequency_encoder = FrequencyEncoder()
minmax_scaler = MinMaxScaler()

# Menggabungkan transformers dalam ColumnTransformer
preprocessor = ColumnTransformer([
    ("ordinal", OrdinalEncoder(), unique_val),
    ("frequency", frequency_encoder, non_unique_val),
    ("scaler", minmax_scaler, numerics.columns)
], remainder = 'passthrough',verbose_feature_names_out=False).set_output(transform='pandas')

# Fit dan transform DataFrame
datatr = preprocessor.fit_transform(data3)
datatr

,Conditions,Name,Details,Release Date,Camera,Selfie,SIM Card,Sensors,Network,Storage,Present price,Selling price
0,1.0,3,2,204,4,4,46,9,25,0.096774,0.068750,1900000.0
23,2.0,3,2,204,4,4,46,9,25,0.096774,0.068750,1900000.0
36,2.0,3,1,204,4,4,46,9,25,0.096774,0.068750,1900000.0
53,0.0,63,26,137,118,315,444,197,141,0.096774,0.359375,6500000.0
54,1.0,4,2,138,4,9,46,4,42,0.000000,0.000000,750000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1283,3.0,24,7,114,56,114,125,114,176,0.483871,0.843781,26449000.0
1284,3.0,24,16,114,56,114,125,114,176,0.483871,0.843781,26449000.0
1285,2.0,32,16,114,56,114,125,114,176,1.000000,0.906281,26595000.0
1286,4.0,24,16,114,56,114,125,114,176,1.000000,0.968781,26595000.0


In [91]:
condition_dict

{'Condition: Fair': 0.0,
 'Condition: Good': 1.0,
 'Condition: Mint': 2.0,
 'Condition: New': 3.0,
 'Condition: Very Good': 4.0}

In [95]:
data3['Conditions'].map(condition_dict)

0       1.0
23      2.0
36      2.0
53      0.0
54      1.0
       ... 
1283    3.0
1284    3.0
1285    2.0
1286    4.0
1287    3.0
Name: Conditions, Length: 869, dtype: float64

In [86]:
datatr.columns

Index(['Conditions', 'Name', 'Details', 'Release Date', 'Camera', 'Selfie',
       'SIM Card', 'Sensors', 'Network', 'Storage', 'Present price',
       'Selling price'],
      dtype='object')

In [60]:
non_unique_val

['Name',
 'Details',
 'Release Date',
 'Camera',
 'Selfie',
 'SIM Card',
 'Sensors',
 'Network']

In [61]:
numerics.columns

Index(['Storage', 'Present price'], dtype='object')

In [44]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('ordinal', OrdinalEncoder(),
                                 Index(['Conditions'], dtype='object')),
                                ('frequency', FrequencyEncoder(),
                                 ['Name', 'Details', 'Release Date', 'Camera',
                                  'Selfie', 'SIM Card', 'Sensors', 'Network']),
                                ('scaler', MinMaxScaler(),
                                 Index(['Storage', 'Present price'], dtype='object'))],
                  verbose_feature_names_out=False)

In [38]:
preprocessor.get_params()['transformers'][2]

('scaler', MinMaxScaler(), Index(['Storage', 'Present price'], dtype='object'))

In [73]:
preprocessor.named_transformers_['scaler'].get_feature_names_out()

array(['Storage', 'Present price'], dtype=object)

In [77]:
preprocessor.named_transformers_['scaler'].data_min_

array([3.200e+01, 2.999e+06])

In [80]:
preprocessor.named_transformers_['scaler'].data_max_

array([1.0240e+03, 3.4999e+07])

In [81]:
storage_dict = {
    'min' : preprocessor.named_transformers_['scaler'].data_min_[0],
    'max' : preprocessor.named_transformers_['scaler'].data_max_[0]
}

price_dict = {
    'min' : preprocessor.named_transformers_['scaler'].data_min_[1],
    'max' : preprocessor.named_transformers_['scaler'].data_max_[1]
}

In [122]:
data3['Camera'][90]

'64 MP PDAF, 8 MP (ultrawide), 5 MP (macro), 5 MP (depth)'

In [123]:
data3['Selfie']

0                                          44 MP
23                                         44 MP
36                                         44 MP
53                                          12MP
54                                          5 MP
                          ...                   
1283    12 MP  f/1.9, 23mm (wide), 1/3.6"", PDAF
1284    12 MP  f/1.9, 23mm (wide), 1/3.6"", PDAF
1285    12 MP  f/1.9, 23mm (wide), 1/3.6"", PDAF
1286    12 MP  f/1.9, 23mm (wide), 1/3.6"", PDAF
1287    12 MP  f/1.9, 23mm (wide), 1/3.6"", PDAF
Name: Selfie, Length: 869, dtype: object

In [105]:
data3['Network'].value_counts()

GSM/CDMA/HSPA/EVDO/LTE/5G              407
GSM / CDMA / HSPA / EVDO / LTE / 5G    176
GSM/CDMA/LTE/HSPA/EVDO                 141
GSM/CDMA/HSPA/EVDO/LTE                  56
GSM/HSPA/LTE                            42
GSM / HSPA / LTE                        25
GSM / HSPA / LTE / 5G                   18
GSM/HSPA/LTE/5G                          2
GSM/CDMA/HSPA/LTE                        1
GSM / LTE / 5G                           1
Name: Network, dtype: int64

In [115]:
data3['SIM Card']

0       Dual SIM (Nano-SIM, dual stand-by)
23      Dual SIM (Nano-SIM, dual stand-by)
36      Dual SIM (Nano-SIM, dual stand-by)
53                         Nano-SIM & eSIM
54      Dual SIM (Nano-SIM, dual stand-by)
                       ...                
1283                     Nano-SIM and eSIM
1284                     Nano-SIM and eSIM
1285                     Nano-SIM and eSIM
1286                     Nano-SIM and eSIM
1287                     Nano-SIM and eSIM
Name: SIM Card, Length: 869, dtype: object

In [118]:
data3['Sensors'][0]

'Fingerprint (under display, optical), accelerometer, gyro, proximity, compass'

In [109]:
data3['Details']

0       8 GB - Stellar Black
23      8 GB - Stellar Black
36             8 GB - Aurora
53                    Purple
54        2 GB - Carbon Gray
                ...         
1283          White Titanium
1284          Black Titanium
1285          Black Titanium
1286          Black Titanium
1287           Blue Titanium
Name: Details, Length: 869, dtype: object

In [108]:
data3['SIM Card'].value_counts()

Nano-SIM & eSIM                                                          444
Nano-SIM                                                                 186
Nano-SIM and eSIM                                                        125
Nano-SIM/eSIM, dual stand-by                                              50
Dual SIM (Nano-SIM, dual stand-by)                                        46
Single SIM (Nano-SIM) or Hybrid Dual SIM (Nano-SIM, dual stand-by)         6
Hybrid Dual SIM (Nano-SIM, dual stand-by)                                  4
Nano-SIM and eSIM or Dual SIM (2 Nano-SIMs and eSIM, dual stand-by)        4
Dual Nano-SIM                                                              3
Single SIM (Nano-SIM) or Dual SIM (2 Nano-SIMs + eSIM, dual stand-by)      1
Name: SIM Card, dtype: int64

In [120]:
data3['Present price']

0        5199000.0
23       5199000.0
36       5199000.0
53      14499000.0
54       2999000.0
           ...    
1283    30000000.0
1284    30000000.0
1285    32000000.0
1286    34000000.0
1287    32000000.0
Name: Present price, Length: 869, dtype: float64

In [119]:
data3['Network']

0                          GSM / HSPA / LTE
23                         GSM / HSPA / LTE
36                         GSM / HSPA / LTE
53                   GSM/CDMA/LTE/HSPA/EVDO
54                             GSM/HSPA/LTE
                       ...                 
1283    GSM / CDMA / HSPA / EVDO / LTE / 5G
1284    GSM / CDMA / HSPA / EVDO / LTE / 5G
1285    GSM / CDMA / HSPA / EVDO / LTE / 5G
1286    GSM / CDMA / HSPA / EVDO / LTE / 5G
1287    GSM / CDMA / HSPA / EVDO / LTE / 5G
Name: Network, Length: 869, dtype: object

In [85]:
np.save('storage_scaler.npy', storage_dict)
np.save('price_scaler.npy', price_dict)

In [89]:
np.load('storage_scaler.npy', allow_pickle=True).item()

{'min': 32.0, 'max': 1024.0}

In [ ]:
preprocessor.named_transformers_['ordinal'].categories_

[array(['Condition: Fair', 'Condition: Good', 'Condition: Mint',
        'Condition: New', 'Condition: Very Good'], dtype=object)]

In [69]:
conditions = preprocessor.named_transformers_['ordinal'].categories_[0]

# Create dictionary with float indices
condition_dict = {condition: float(index) for index, condition in enumerate(conditions)}
condition_dict

{'Condition: Fair': 0.0,
 'Condition: Good': 1.0,
 'Condition: Mint': 2.0,
 'Condition: New': 3.0,
 'Condition: Very Good': 4.0}

In [70]:
np.save('ordinal_encoder.npy', condition_dict)

In [66]:
np.save('freq_encoder.npy', (preprocessor.named_transformers_['frequency'].frequency_dict))

In [99]:
data3['Name']

0                   Reno6
23                  Reno6
36                  Reno6
53              iPhone 11
54               Redmi 9A
              ...        
1283    iPhone 15 Pro Max
1284    iPhone 15 Pro Max
1285        iPhone 15 Pro
1286    iPhone 15 Pro Max
1287        iPhone 15 Pro
Name: Name, Length: 869, dtype: object

In [102]:
data3['Name'].map((preprocessor.named_transformers_['frequency'].frequency_dict)['Name'])

0        3
23       3
36       3
53      63
54       4
        ..
1283    24
1284    24
1285    32
1286    24
1287    32
Name: Name, Length: 869, dtype: int64

In [63]:
(preprocessor.named_transformers_['frequency'].frequency_dict)

{'Name': {'iPhone 13': 74,
  'iPhone 14 Pro Max': 63,
  'iPhone 11': 63,
  'iPhone 13 Pro Max': 50,
  'iPhone 13 Pro': 49,
  'iPhone 12': 48,
  'iPhone 11 Pro Max': 42,
  'iPhone 12 Pro Max': 42,
  'iPhone 12 Pro': 38,
  'iPhone 11 Pro': 32,
  'iPhone 15 Pro': 32,
  'iPhone 14': 30,
  'iPhone 14 Pro': 30,
  'iPhone 15': 30,
  'iPhone 15 Plus': 28,
  'iPhone XR': 24,
  'iPhone 15 Pro Max': 24,
  'iPhone X': 21,
  'iPhone 7 Plus': 15,
  'iPhone 14 Plus': 13,
  'iPhone XS Max': 11,
  'Galaxy Z Fold3 5G': 8,
  'iPhone 13 Mini': 8,
  'Galaxy Z Flip 5': 7,
  'iPhone 8': 6,
  'C33': 5,
  'Galaxy Z Fold 5': 4,
  'Redmi 10 5G': 4,
  'iPhone 8 Plus': 4,
  'Galaxy A32': 4,
  'iPhone XS': 4,
  'Redmi 9A': 4,
  'iPhone SE (2nd generation)': 4,
  'Galaxy Z Flip3 5G': 3,
  'Reno6': 3,
  'Y36': 3,
  'Y22': 3,
  'Galaxy A14': 3,
  'A77s': 3,
  'Y27': 3,
  'Y27 5G': 3,
  'Poco M4 Pro': 2,
  'iPhone 7': 2,
  'Galaxy A34 5G': 2,
  'V25 5G': 2,
  'V25e': 2,
  'Reno5 4G': 1,
  'Poco M3 Pro 5G': 1,
  'Galaxy